In [2]:
### Predict Messages
from langchain.chat_models import ChatOpenAI
from  langchain.schema  import  HumanMessage, AIMessage, SystemMessage

chat = ChatOpenAI(temperature=0.1) ## call Chat large language models API.

messages  = [
    SystemMessage(content='you are a geography expert. and you only reply in korean.'),
    AIMessage(content='무엇이 궁금하신가요? 저는 지리학 박사입니다!'),
    HumanMessage(content='한국과 일본의 지리적 차이에 대해 알려줘.')
]

# Pass a message sequence to the model and return a message prediction.
chat.predict_messages(messages)

AIMessage(content='한국과 일본은 지리적으로 많은 차이가 있습니다. 한국은 한반도에 위치하고 있으며, 주변에는 동해와 서해, 그리고 북한과 중국이 인접해 있습니다. 한국은 산이 많고 지형이 고르지 않아서 농업이 어려운 면이 있습니다. \n\n반면에 일본은 일본 제도로 이루어져 있으며, 주변에는 동해와 태평양이 인접해 있습니다. 일본은 화산 활동이 활발하고, 섬으로 이루어져 있어서 지형이 다양하고 복잡합니다. 이러한 지형적 특성으로 인해 농업과 어업이 발달하였습니다. \n\n또한, 한국과 일본은 역사적으로도 다양한 영향을 받아왔고, 문화적으로도 차이가 있습니다. 이러한 지리적, 역사적, 문화적 차이로 두 나라는 각자 독특한 매력을 지니고 있습니다.')

In [3]:
### Prompt Templates
from langchain.prompts import PromptTemplate, ChatPromptTemplate 

# Load a prompt template from a template.
template = PromptTemplate.from_template('{country_a}과 {country_b}의 지리적 차이에 대해 알려줘.')

prompt = template.format(
country_a='korea',
country_b='japan')

# Pass a single string input to the model and return a string prediction.
chat.predict(prompt) 

# Create a chat prompt template from a variety of message format그냥,, s.
template = ChatPromptTemplate.from_messages ([
    ('system', 'you are a geography expert. and you only reply in {language}.'),
    ('ai', '무엇이 궁금하신가요? 저는 {name}입니다!'),
    ('human', '{country_a}과 {country_b}의 지리적 차이에 대해 알려줘.')
])

prompt = template.format_messages(language="korean", name='배다롱', country_a='한국', country_b='일본')

#Pass a message sequence to the model and return a message prediction.
chat.predict_messages(prompt) 

AIMessage(content='한국과 일본은 두 나라 모두 동아시아 지역에 위치하고 있습니다. 한국은 한반도에 위치하고 있으며, 북쪽으로는 북한과 국경을 이루고 있습니다. 일본은 일본 제도로 이루어져 있으며, 본토와 여러 섬으로 구성되어 있습니다. 또한, 한국은 대륙과 인접해 있지만, 일본은 섬나라로서 바다에 둘러싸여 있습니다. 두 나라의 지형은 산악 지대와 해안선이 많이 있어서 자연 경관이 다양하고 아름다운 특징을 가지고 있습니다.')

In [4]:
from langchain.schema import BaseOutputParser

# Output parsers help structure language model responses.
# My personal output parser
class CommaOutputParser(BaseOutputParser):
    # Parse a single string model output into some structure.
    def parse(self,text): 
        items= text.strip().split(',')
        return list(map(str.strip, items))
    
    
p = CommaOutputParser()
p.parse("hello, how, are, you?")    
# ['hello', 'how', 'are', 'you?']

['hello', 'how', 'are', 'you?']

In [5]:

template = ChatPromptTemplate.from_messages([
    ("system", "you are a list generating machine. everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. do not reply with anything else."),
    ("human", "{question}"),
])

# prompt = template.format_messages(max_items=10, question = 'what are the colors?')
# results = chat.predict_messages(prompt)
# p = CommaOutputParser()
# p.parse(results.content)

#LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together. 
# | : Compose this runnable with another object to create a RunnableSequence.
# chain organized with template | language Model | Output parser
chain = template | chat | CommaOutputParser()
chain.invoke({"max_items":5, "question": "flowers"})

['rose', 'daisy', 'tulip', 'sunflower', 'lily']